In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model

import tensorflow as tf

In [2]:
def n_mode_product(x, u, n):# x, uはデータ型がfloat32である必要がある
    n = int(n)
    # We need one letter per dimension
    # (maybe you could find a workaround for this limitation)
    if n > 26:
        raise ValueError('n is too large.')
    ind = ''.join(chr(ord('a') + i) for i in range(n))
    exp = f'{ind}K...,JK->{ind}J...'
    return tf.einsum(exp, x, u)

In [72]:
class TeSIA2(object):
    def __init__(self, random_state = 0):
        self.random_state = random_state
    
    
    def fit(self, X, y, fit_intercept=True, normalize=False, copy_X=True, n_jobs=1, alternate_num = 5):
        if len(y.shape) == 1:
            self.y = y.reshape(-1, 1)
        else:
            self.y = y
        
        #交互最適を止めるための検証用データ作成
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X, y, test_size=0.2, random_state=self.random_state)
        self.val_score = -float('inf')
        
        #企業データの特徴量の次元数
        feature_len_train = self.X_train.shape[0]
        feature_len_com = X.shape[1] 
        feature_len_eve = X.shape[2]
        feature_len_sen = X.shape[3]
        
        # それぞれのデータの係数
        self.W_com = np.random.rand(feature_len_com, 1)
        self.W_eve = np.random.rand(feature_len_eve, 1)
        self.W_sen = np.random.rand(feature_len_sen, 1)
        self.b = np.random.rand(1, 1)
        
        # 後のfor文で数字とそれぞれの変数を認識するための辞書
        dic_len = {1:feature_len_com, 2:feature_len_eve, 3:feature_len_sen}
        dic_W = {1:self.W_com, 2:self.W_eve, 3:self.W_sen} 
        list_W = ['com', 'eve', 'sen']
        print(self.X_train)
        for num in range(alternate_num):
            print(str(num+1)+"回目")
            for m in [1,2,3]: # １：企業、２：イベント、３：センチメント
                print()
                print('\n'+['企業', 'イベント', 'センチメント'][m-1])
                self.model = linear_model.LinearRegression(fit_intercept=fit_intercept, normalize=normalize, copy_X=copy_X, n_jobs=n_jobs)
                exec("self.X = n_mode_product(n_mode_product(self.X_train, self.W_{0}.T, 3-m//3), self.W_{1}.T, 2-m//2)".format(list_W[2-m//3], list_W[1-m//2]))
                exec("self.X = tf.reshape(self.X ,[feature_len_train, feature_len_{}]).numpy()".format(list_W[m-1]))
                self.model.fit(self.X, self.y_train)
                
                exec("self.W_{0} = self.model.coef_.reshape([feature_len_{0},1])".format(list_W[m-1]))
                self.b = self.model.intercept_
                print('\n企業 : {}'.format(self.W_com))
                print('イベント : {}'.format(self.W_eve))
                print('センチメント : {}'.format(self.W_sen))
                print('intercept : {}'.format(self.b))
                print('='*50)
            #交互最適化を止めるか判断
            if (self.score(self.X_test, self.y_test) -self.val_score)/self.val_score<0.01:
                self.num = num
                return self
            self.val_score = self.score(self.X_test, self.y_test)
        self.num = num
        return self
    
    def predict(self, X, y=None):
        # tf.transpose()で置換されている
        y_pred = n_mode_product(n_mode_product(n_mode_product(X, tf.transpose(self.W_com), 1), tf.transpose(self.W_eve), 2), tf.transpose(self.W_sen), 3) + self.b
        y_pred = tf.reshape(y_pred,[len(X), 1]).numpy()
        
        return y_pred
    
    def coef_(self):
        result_W_com = self.W_com
        result_W_eve = self.W_eve
        result_W_sen = self.W_sen
        
        return result_W_com, result_W_eve, result_W_sen
    
    def intercept_(self):
        result_b = self.b
        
        return result_b
    
    def score(self, X, y):
        
        
        if len(y.shape) == 1:
            print('変換')
            y = y.reshape(-1, 1)
        # tf.transpose()で置換されている
        y_pred = n_mode_product(n_mode_product(n_mode_product(X, tf.transpose(self.W_com), 1), tf.transpose(self.W_eve), 2), tf.transpose(self.W_sen), 3) + self.b
        y_pred = tf.reshape(y_pred,[len(X), 1]).numpy()
        u = ((y_pred - y)**2).sum()
        v = ((y - y.mean())**2).sum()
        R2 = 1 - u/v
        
        return R2

In [73]:
W_com = np.array([[1], [2], [-1]])
W_eve = np.array([[-1], [1], [1]])
W_sen = np.array([[3], [1], [3]])
b = np.array([2])

In [74]:
b.shape

(1,)

In [75]:
X = np.random.rand(10000,3, 3, 3)
X_test = np.random.rand(1000,3, 3, 3)
X

array([[[[8.73183405e-01, 8.79222938e-01, 7.79222370e-01],
         [5.71547039e-02, 1.95638770e-01, 3.78067927e-01],
         [6.61139738e-01, 7.37977718e-01, 7.51805310e-01]],

        [[4.95466622e-01, 9.00576398e-01, 1.88919843e-01],
         [5.80819271e-01, 6.02559427e-01, 3.75446653e-01],
         [1.58063217e-01, 8.70001284e-01, 7.58292110e-01]],

        [[3.63682822e-02, 2.55602915e-01, 9.76018835e-02],
         [7.00799105e-01, 2.63747558e-02, 9.18582253e-01],
         [1.89454671e-01, 1.28424719e-01, 7.62272414e-01]]],


       [[[7.54053778e-01, 1.80717946e-01, 5.57743120e-01],
         [2.95173378e-01, 1.62785112e-01, 7.59374340e-01],
         [8.71858898e-01, 4.80999289e-01, 9.34637151e-01]],

        [[2.68894979e-01, 6.96408516e-01, 7.66219581e-01],
         [7.20561664e-02, 4.95846017e-01, 1.60000869e-01],
         [8.58257948e-01, 4.34057120e-01, 5.00080106e-01]],

        [[4.09967278e-01, 9.27436405e-02, 3.79245771e-03],
         [7.88044177e-02, 1.96364993e-01, 2.

In [76]:
X = np.random.randn(10000,3, 3, 3)
X_test = np.random.randn(1000,3, 3, 3)
X

array([[[[-1.94926507e+00,  1.67049544e+00, -6.46821793e-01],
         [-8.42007628e-01, -9.68979578e-01, -1.09557796e+00],
         [-5.07698237e-01, -4.26933277e-01,  3.26139253e-01]],

        [[ 8.88907646e-01, -4.38207888e-01,  3.80635935e-01],
         [ 3.63815896e-01,  1.10450827e+00, -2.17870575e+00],
         [ 1.75116387e+00, -5.31303961e-01,  6.89137554e-01]],

        [[-5.25120876e-02, -3.70602296e-01, -9.64121112e-01],
         [ 2.18498270e+00, -5.78182104e-01, -2.86632619e-01],
         [ 1.96456773e+00,  5.99610293e-01, -2.70063796e-01]]],


       [[[ 1.86557685e-01,  1.22074774e+00, -6.11022263e-01],
         [-1.14497067e+00,  4.91971296e-01, -9.64103610e-02],
         [ 8.55629614e-01,  4.47693596e-01,  7.49760250e-01]],

        [[ 4.67287674e-01, -9.70226743e-01, -7.16238873e-01],
         [ 1.02971163e-01,  5.93084351e-01, -4.96685982e-01],
         [ 9.30094848e-01, -3.58572968e-01,  4.66283786e-01]],

        [[ 6.78235630e-01, -9.15206533e-01,  1.34044675e+0

In [77]:
y = (n_mode_product(n_mode_product(n_mode_product(X, W_com.T, 1), W_eve.T, 2), W_sen.T, 3) + b).numpy().reshape(len(X), 1)
y_test = (n_mode_product(n_mode_product(n_mode_product(X_test, W_com.T, 1), W_eve.T, 2), W_sen.T, 3) + b).numpy().reshape(len(X_test), 1)
y

array([[-15.69804121],
       [ 10.6917825 ],
       [ 27.94216287],
       ...,
       [ 15.86447234],
       [-18.70106003],
       [  1.97367809]])

In [78]:
model = TeSIA2()

In [79]:
model.fit(X, y, alternate_num = 1)

[[[[ 5.46334890e-01  9.78764973e-01  5.90208651e-01]
   [-3.85562737e-01  1.22762835e-01 -2.46336132e+00]
   [-1.31318230e+00 -3.57458409e-01 -4.52595104e-01]]

  [[-1.11443803e+00  1.35172659e+00 -1.56530112e+00]
   [-3.00176390e-01  1.59814016e+00 -1.29515869e+00]
   [-3.12314686e-01  1.73836077e+00 -6.05260059e-02]]

  [[ 8.92363296e-01  2.40997774e-01  2.35374858e+00]
   [ 1.14300688e+00 -6.92042061e-01  7.49954147e-02]
   [-7.91543186e-01  1.85084787e-01  4.87041995e-01]]]


 [[[-1.06368867e+00  2.98779703e-02  9.74562934e-01]
   [-4.23843108e-01  2.43724422e+00 -6.96037883e-01]
   [ 1.42104732e+00  7.27840022e-01 -2.75933465e-01]]

  [[ 1.27428338e+00  8.71508098e-02  1.27687989e-01]
   [ 5.01454884e-01 -4.11482577e-01 -2.13376987e+00]
   [-7.16284628e-01  1.18623600e+00 -6.19995187e-01]]

  [[-3.40100124e-01 -2.42437990e-01 -9.00687786e-01]
   [ 1.57583145e+00  1.04277620e+00  8.01362748e-02]
   [-1.27087050e-01  3.01084993e-01  1.36587500e+00]]]


 [[[ 1.15315832e+00 -4.9298953

C:\Users\Tomohiro\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in double_scalars


In [80]:
# テストデータの精度を計算
print("テストデータにフィット")
print("テストデータの精度 =", model.score(X_test, y_test))
print()
print("※参考")
print("訓練データの精度 =", model.score(X, y))

テストデータにフィット
テストデータの精度 = 0.9996403159553784

※参考
訓練データの精度 = 0.9995924073832407
